In [ ]:
# Download: https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

In [1]:
import os
# print(os.getcwd())
# print(os.listdir())
os.path.isfile('./aclImdb_v1.tar.gz')

True

In [ ]:
# import tarfile
# tar = tarfile.open('./aclImdb.tar.gz', 'r')
# tar.extractall('')
# tar.close()

In [2]:
import glob

raw_trains = glob.glob('./aclImdb/train/pos/*.txt')
len(raw_trains)

12500

In [3]:
raw_trains[0]

'./aclImdb/train/pos\\0_9.txt'

In [4]:
# %%time
# def read_text_files(directory):
#     texts = []
#     labels = []
#     for sentiment in ['pos', 'neg']:
#         sentiment_dir = os.path.join(directory, sentiment) 
#         for filepath in glob.iglob(os.path.join(sentiment_dir, '*.txt')):
#             with open(filepath, 'r', encoding='utf-8') as file:
#                 if len(texts) < 1000:   
#                     texts.append(file.read())
#                     labels.append(1 if sentiment == 'pos' else 0)
#     return texts, labels

# train_texts, train_labels = read_text_files('./aclImdb/train/')
# test_texts, test_labels = read_text_files('./aclImdb/test/')
# print(len(train_texts), len(test_texts))

In [5]:
%%time
import os
import random

def read_text_files(directory, num_samples=1000):
    texts = []
    labels = []
    for sentiment in ['pos', 'neg']:
        sentiment_dir = os.path.join(directory, sentiment)
        files = glob.glob(os.path.join(sentiment_dir, '*.txt'))
        
        # Select random files
        random_files = random.sample(files, min(num_samples, len(files)))
        
        for filepath in random_files:
            with open(filepath, 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(1 if sentiment == 'pos' else 0)
    return texts, labels

train_texts, train_labels = read_text_files('./aclImdb/train/')
test_texts, test_labels = read_text_files('./aclImdb/test/')

Wall time: 10.1 s


<span style="color: gray; font-family: Helvetica; font-size: 30px;">Wall time vs. CPU time:</span>

### Wall time
    The total time taken to complete a process from beginning to end, including all waiting times. On the other hand, CPU time represents the actual time the CPU spends executing a process, excluding any waiting periods for I/O operations or other processes. (baking analogy: gathering the ingredients until taking the cake out-> Wall time)

    In a multiprocessor environment, CPU time can surpass wall time due to parallel execution. Wall time provides a sense of the total duration your program will run, while CPU time shows how much processing power your program utilizes. Both these times are essential to measure for understanding program performance.

In [6]:
type(train_texts)

list

In [7]:
import sys

print("Size of train_texts in bytes: ", sys.getsizeof(train_texts))

total_size = sys.getsizeof(train_texts) + sum(sys.getsizeof(i) for i in train_texts)

total_size_kb = total_size / 1024
total_size_mb = total_size_kb / 1024

print(f"Total size of train_texts in kilobytes: ", "{:.2f}".format(total_size_kb))
print("Total size of train_texts in megabytes: ", total_size_mb)

Size of train_texts in bytes:  16552
Total size of train_texts in kilobytes:  2652.63
Total size of train_texts in megabytes:  2.5904550552368164


<span style="color: gray; font-family: Helvetica; font-size: 30px;"> Getting the size of objects in Python:
 
### sys.getsizeof(train_texts)
     returns the size of the train_texts object itself, which includes the overhead of the list object and any additional attributes or metadata associated with it. It does not take into account the size of the elements contained within the list.

### total_size
    calculated by summing the size of train_texts obtained from sys.getsizeof(train_texts) with the size of each individual element within train_texts obtained using sys.getsizeof(i) in a generator expression. This provides a more accurate estimation of the total size, considering the memory used by both the list object and its elements.

<span style="color: gray; font-family: Helvetica; font-size: 30px;">Formatting floating-point numbers

### {}: 
    These curly braces are used to enclose the field that will be replaced with the formatted value
    
### :: 
    It indicates the start of the format specification.
    
### .3f: 
    This is the format specifier itself. It consists of two parts:
    
### .: 
    The dot specifies the precision or the number of decimal places.
    
### 3f: 
    The f is the type specifier for a floating-point number, and 3 represents the number of decimal places.

<span style="color: gray; font-family: Helvetica; font-size: 30px;"> Preporcessing:

<span style="color: black; font-family: Helvetica; font-size: 15px;"> To clean and standardize the input, preprocessing is a crucial step when working with text data.

### Tokenization: 
    This is the process of breaking down the text into individual words or tokens. We use nltk.tokenize.word_tokenize for this purpose. This function splits the text by whitespace and punctuation to create a list of individual words.

### Lower casing: 
    We convert all the text to lower case. This is done to ensure that the same word in different cases (for example, "Movie" and "movie") is treated as the same word.

### Punctuation Removal: 
    This step removes all punctuation from the text. Punctuation doesn't usually add much meaning to the text and can be removed. We check if each token is a punctuation symbol using string.punctuation.

### Stop words Removal: 
    Stop words are common words like 'is', 'the', 'and', etc. that do not carry important meaning and are often removed from the text. We use a predefined list of stop words from NLTK, which can be accessed using nltk.corpus.stopwords.words('english').

### Lemmatization: 
    This is the process of reducing a word to its base or root form. For example, 'running' is transformed to 'run', 'better' is changed to 'good'. It is a more advanced form of stemming because it considers the context and part of speech of the word. We use WordNetLemmatizer from NLTK for lemmatization.

In [41]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
import numpy as np

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Xamani\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Xamani\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Xamani\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
string.punctuation = string.punctuation + "''"
def preprocess_text(text):
    # Tokenize and lower case
    tokens = word_tokenize(text.lower())
    
    # Remove punctuation and filter out stopwords, then lemmatize
    processed_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in string.punctuation and token not in stop_words]
    
    return processed_tokens

In [10]:
train_tokens = [preprocess_text(text) for text in train_texts]
test_tokens = [preprocess_text(text) for text in test_texts]

In [11]:
print(len(train_tokens))

2000


In [12]:
print(train_texts[0])

Jim Carrey is good as usual, and even though there are quite a few "Jim Carrey moments", it's definitely not a "Jim Carrey movie".<br /><br />It's targeted mostly at children, and I managed to enjoy it as such movie. It was promoted in Israel as another Jim Carrey movie, so those who expected a weird over the top comedy were disappointed.<br /><br />The movie has nice moments and works well as a movie for kids. I can't say I LOVED this movie, but then again I'm not its target audience!


In [13]:
print(train_texts[0].lower())

jim carrey is good as usual, and even though there are quite a few "jim carrey moments", it's definitely not a "jim carrey movie".<br /><br />it's targeted mostly at children, and i managed to enjoy it as such movie. it was promoted in israel as another jim carrey movie, so those who expected a weird over the top comedy were disappointed.<br /><br />the movie has nice moments and works well as a movie for kids. i can't say i loved this movie, but then again i'm not its target audience!


In [14]:
print(word_tokenize(train_texts[0].lower()))

['jim', 'carrey', 'is', 'good', 'as', 'usual', ',', 'and', 'even', 'though', 'there', 'are', 'quite', 'a', 'few', '``', 'jim', 'carrey', 'moments', "''", ',', 'it', "'s", 'definitely', 'not', 'a', '``', 'jim', 'carrey', 'movie', "''", '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'it', "'s", 'targeted', 'mostly', 'at', 'children', ',', 'and', 'i', 'managed', 'to', 'enjoy', 'it', 'as', 'such', 'movie', '.', 'it', 'was', 'promoted', 'in', 'israel', 'as', 'another', 'jim', 'carrey', 'movie', ',', 'so', 'those', 'who', 'expected', 'a', 'weird', 'over', 'the', 'top', 'comedy', 'were', 'disappointed.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'the', 'movie', 'has', 'nice', 'moments', 'and', 'works', 'well', 'as', 'a', 'movie', 'for', 'kids', '.', 'i', 'ca', "n't", 'say', 'i', 'loved', 'this', 'movie', ',', 'but', 'then', 'again', 'i', "'m", 'not', 'its', 'target', 'audience', '!']


In [15]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\'\''

In [16]:
print([lemmatizer.lemmatize(token) for token in word_tokenize(train_texts[0].lower()) if token not in string.punctuation and token not in stop_words])

['jim', 'carrey', 'good', 'usual', 'even', 'though', 'quite', '``', 'jim', 'carrey', 'moment', "'s", 'definitely', '``', 'jim', 'carrey', 'movie', 'br', 'br', "'s", 'targeted', 'mostly', 'child', 'managed', 'enjoy', 'movie', 'promoted', 'israel', 'another', 'jim', 'carrey', 'movie', 'expected', 'weird', 'top', 'comedy', 'disappointed.', 'br', 'br', 'movie', 'nice', 'moment', 'work', 'well', 'movie', 'kid', 'ca', "n't", 'say', 'loved', 'movie', "'m", 'target', 'audience']


In [17]:
string.punctuation = string.punctuation + "--"

In [18]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\'\'--'

In [19]:
print([lemmatizer.lemmatize(token) for token in word_tokenize(train_texts[0].lower()) if token not in string.punctuation and token not in stop_words])

['jim', 'carrey', 'good', 'usual', 'even', 'though', 'quite', '``', 'jim', 'carrey', 'moment', "'s", 'definitely', '``', 'jim', 'carrey', 'movie', 'br', 'br', "'s", 'targeted', 'mostly', 'child', 'managed', 'enjoy', 'movie', 'promoted', 'israel', 'another', 'jim', 'carrey', 'movie', 'expected', 'weird', 'top', 'comedy', 'disappointed.', 'br', 'br', 'movie', 'nice', 'moment', 'work', 'well', 'movie', 'kid', 'ca', "n't", 'say', 'loved', 'movie', "'m", 'target', 'audience']


In [20]:
print([lemmatizer.lemmatize(token.replace('.', '')) for token in word_tokenize(train_texts[0].lower()) if token not in string.punctuation and token not in stop_words])

['jim', 'carrey', 'good', 'usual', 'even', 'though', 'quite', '``', 'jim', 'carrey', 'moment', "'s", 'definitely', '``', 'jim', 'carrey', 'movie', 'br', 'br', "'s", 'targeted', 'mostly', 'child', 'managed', 'enjoy', 'movie', 'promoted', 'israel', 'another', 'jim', 'carrey', 'movie', 'expected', 'weird', 'top', 'comedy', 'disappointed', 'br', 'br', 'movie', 'nice', 'moment', 'work', 'well', 'movie', 'kid', 'ca', "n't", 'say', 'loved', 'movie', "'m", 'target', 'audience']


In [21]:
import re
print([lemmatizer.lemmatize(re.sub(r"\.{1,}", "", token)) for token in word_tokenize(train_texts[0].lower()) if token not in string.punctuation and token not in stop_words])

['jim', 'carrey', 'good', 'usual', 'even', 'though', 'quite', '``', 'jim', 'carrey', 'moment', "'s", 'definitely', '``', 'jim', 'carrey', 'movie', 'br', 'br', "'s", 'targeted', 'mostly', 'child', 'managed', 'enjoy', 'movie', 'promoted', 'israel', 'another', 'jim', 'carrey', 'movie', 'expected', 'weird', 'top', 'comedy', 'disappointed', 'br', 'br', 'movie', 'nice', 'moment', 'work', 'well', 'movie', 'kid', 'ca', "n't", 'say', 'loved', 'movie', "'m", 'target', 'audience']


<span style="color: gray; font-family: Helvetica; font-size: 30px;"> Compute Term Frequencies (TF)

    In this step we calculate how often each word appears in each document. We start by initializing an empty dictionary tf_dict. Then for each token in the list of tokens, we check if it's already in tf_dict. If it is, we increment its count by one. If it's not, we add it to tf_dict with a count of one. In the end, the keys will be the unique words in the tokens, and the values will be the counts of each word.

    The list comprehension at the end applies this function to each list of tokens in train_tokens and test_tokens, giving us the term frequencies for each document in our train and test sets.

In [22]:
def compute_tf(tokens):
    tf_dict = {}
    for token in tokens:
        if token in tf_dict:
            tf_dict[token] += 1
        else:
            tf_dict[token] = 1
    return tf_dict

train_tf = [compute_tf(tokens) for tokens in train_tokens]
test_tf = [compute_tf(tokens) for tokens in test_tokens]


In [23]:
print(train_tf[0])

{'jim': 4, 'carrey': 4, 'good': 1, 'usual': 1, 'even': 1, 'though': 1, 'quite': 1, '``': 2, 'moment': 2, "'s": 2, 'definitely': 1, 'movie': 6, 'br': 4, 'targeted': 1, 'mostly': 1, 'child': 1, 'managed': 1, 'enjoy': 1, 'promoted': 1, 'israel': 1, 'another': 1, 'expected': 1, 'weird': 1, 'top': 1, 'comedy': 1, 'disappointed.': 1, 'nice': 1, 'work': 1, 'well': 1, 'kid': 1, 'ca': 1, "n't": 1, 'say': 1, 'loved': 1, "'m": 1, 'target': 1, 'audience': 1}


In [24]:
print(sorted(train_tf[0].items(), key=lambda x: x[1], reverse=True))

[('movie', 6), ('jim', 4), ('carrey', 4), ('br', 4), ('``', 2), ('moment', 2), ("'s", 2), ('good', 1), ('usual', 1), ('even', 1), ('though', 1), ('quite', 1), ('definitely', 1), ('targeted', 1), ('mostly', 1), ('child', 1), ('managed', 1), ('enjoy', 1), ('promoted', 1), ('israel', 1), ('another', 1), ('expected', 1), ('weird', 1), ('top', 1), ('comedy', 1), ('disappointed.', 1), ('nice', 1), ('work', 1), ('well', 1), ('kid', 1), ('ca', 1), ("n't", 1), ('say', 1), ('loved', 1), ("'m", 1), ('target', 1), ('audience', 1)]


In [27]:
from collections import Counter

def compute_tf(tokens):
    return Counter(tokens)

train_tf = [compute_tf(tokens) for tokens in train_tokens]
test_tf = [compute_tf(tokens) for tokens in test_tokens]

In [28]:
print(train_tf[0])

Counter({'movie': 6, 'jim': 4, 'carrey': 4, 'br': 4, '``': 2, 'moment': 2, "'s": 2, 'good': 1, 'usual': 1, 'even': 1, 'though': 1, 'quite': 1, 'definitely': 1, 'targeted': 1, 'mostly': 1, 'child': 1, 'managed': 1, 'enjoy': 1, 'promoted': 1, 'israel': 1, 'another': 1, 'expected': 1, 'weird': 1, 'top': 1, 'comedy': 1, 'disappointed.': 1, 'nice': 1, 'work': 1, 'well': 1, 'kid': 1, 'ca': 1, "n't": 1, 'say': 1, 'loved': 1, "'m": 1, 'target': 1, 'audience': 1})


<span style="color: gray; font-family: Helvetica; font-size: 30px;"> Compute Inverse Document Frequency (IDF)

    In this step we compute how many documents contain each word. Then, use this to compute the IDF for each word. We compute the Inverse Document Frequency (IDF) for each unique word across all documents. IDF gives more weight to the words that are less frequent in the entire corpus (set of all documents) which often are more informative than the frequently occurring words. It can be calculated as the log of the total number of documents divided by the number of documents containing the word. The purpose of using the logarithm is to dampen the effect of the idf; without it, the idf would give too much weight to very rare words.
    

In [29]:
import math

def compute_idf(documents):
    idf_dict = dict()
    total_docs = len(documents)

    # Count number of documents that contain each word
    for doc in documents:
        for word in set(doc):  # Using set to remove duplicates in each document
            if word in idf_dict:
                idf_dict[word] += 1
            else:
                idf_dict[word] = 1

    # Compute IDF for each word
    for word, val in idf_dict.items():
        idf_dict[word] = math.log(total_docs / float(val))
    
    return idf_dict

idf = compute_idf(train_tokens)


In [30]:
print(len(idf))

26959


In [31]:
idf['underrated']

4.961845129926823

    In this function, we first create an empty dictionary idf_dict. Then for each document in documents (which are our tokenized and preprocessed training documents), we iterate over the set of unique words in the document. If a word is in idf_dict, we increment its value by one (because we found another document containing the word). If a word is not in idf_dict, we add it with a value of one.

In [32]:
from collections import defaultdict
import math

def compute_idf(documents):
    idf = defaultdict(int)
    total_docs = len(documents)

    for doc in documents:
        for word in set(doc):
            idf[word] += 1

    for word, val in idf.items():
        idf[word] = math.log(total_docs / float(val))
    
    return idf

idf = compute_idf(train_tokens)



<span style="color: gray; font-family: Helvetica; font-size: 30px;"> Compute TF-IDF
    
    Finally, we can compute the TF-IDF for each word in each document. The TF-IDF is simply the product of the TF and IDF and stands for Term Frequency-Inverse Document Frequency. It is numerical statistic used to reflect how important a word is to a document in a collection or corpus. It's a commonly used technique for transforming text into meaningful representations, which can be used for various data mining and machine learning tasks.

In [33]:
def compute_tfidf(tf, idf):
    tfidf = {}
    for word, val in tf.items():
        if word in idf:
            tfidf[word] = val * idf[word]
        else:
            tfidf[word] = 0.0  # Assign a default value for unseen words
    return tfidf

train_tfidf = [compute_tfidf(tf, idf) for tf in train_tf]
test_tfidf = [compute_tfidf(tf, idf) for tf in test_tf]

In [38]:
test_tfidf[0]

{'actor': 1.6528674703614363,
 'dark': 3.158251203051766,
 'film': 3.2631928670651456,
 'truly': 2.995732273553991,
 'believable': 3.490028595368771,
 'well': 1.2928040180325517,
 'cast': 2.132842318406951,
 'quality': 3.2064533048696435,
 'camera': 2.864704011147587,
 'work': 1.7573580425107227,
 'make': 2.1842666451408013,
 'feel': 2.067512970814562,
 'screenplay': 3.912023005428146,
 'intense': 4.767689115485866,
 'wander': 5.991464547107982,
 'plot': 1.6270928476728213,
 'one': 1.1312677205219714,
 'want': 1.810942288644829,
 'watch': 3.0056563547516846,
 'second': 2.610469872763346,
 'time': 0.9675840262617057,
 'new': 2.107841016201534,
 'perspective': 4.4654082436129325,
 'gained': 6.502290170873972,
 'ending': 2.583622622727158,
 'showed': 4.017383521085972,
 'small': 2.796881414808826,
 'group': 6.437751649736401,
 'patron': 15.201804919084164,
 'gadsden': 0.0,
 "'s": 0.3516874024276933,
 'center': 4.268697949366879,
 'cultural': 5.035953102080546,
 'art': 3.2314546070750607,


### TF 
    (Term Frequency) is a measure of how frequently a term occurs in a document. In the simplest case, it's the raw count of a term in a document. If a word occurs frequently in a document, it's likely to be more important to the document.

### IDF (Inverse Document Frequency) 
    is a measure of how important a term is across the entire corpus (the entire collection of all documents). The intuition here is that words that appear in many different documents are likely less meaningful (i.e., they're probably common words like "the" or "and"), so they're given lower weight. IDF is computed as the logarithm of the number of total documents divided by the number of documents containing the word.
    
    By multiplying these two numbers together, we get the TF-IDF score for a word in a document. The result is that words that are common in a single document but rare across other documents have a high TF-IDF score.

In [42]:
def compute_idf(documents, max_features=5000):
    idf_dict = {}
    total_docs = len(documents)
    word_counts = {}

    for doc in documents:
        for word in set(doc): 
            if word in idf_dict:
                idf_dict[word] += 1
            else:
                idf_dict[word] = 1
                
            # Count frequency of each word
            if word in word_counts:
                word_counts[word] += 1
            else:
                word_counts[word] = 1

    # max_features with highest document frequency
    top_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)[:max_features]

    # Compute IDF for each top word
    for word, _ in top_words:
        idf_dict[word] = math.log(total_docs / float(idf_dict[word]))
    
    # Only keep the words, not their frequencies
    top_words = [word for word, _ in top_words]
    
    return idf_dict, top_words

idf, vocabulary = compute_idf(train_tokens)

In [43]:
def normalize_tfidf(tfidf_dicts, vocabulary):
    normalized_tfidf = []

    for tfidf in tfidf_dicts:
        # Initialize a vector for this document
        tfidf_vector = [0] * len(vocabulary)
        
        # Compute L2 norm
        l2_norm = np.sqrt(sum([value**2 for value in tfidf.values()]))

        # Normalize tf-idf values and add them to the vector
        for word, value in tfidf.items():
            if word in vocabulary:
                index = vocabulary.index(word)
                tfidf_vector[index] = value / l2_norm
                
        normalized_tfidf.append(tfidf_vector)
    
    return normalized_tfidf

train_tfidf_normalized = normalize_tfidf(train_tfidf, vocabulary)
test_tfidf_normalized = normalize_tfidf(test_tfidf, vocabulary)

In [45]:
train_tfidf_normalized[0]

[0.018386908794721084,
 0.06975342084295526,
 0.01249632573114062,
 0,
 0.057048995461369406,
 0,
 0,
 0.04042824441264822,
 0,
 0.02577963248462714,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.029745118782690653,
 0,
 0,
 0.03379516781796117,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.04110993561456676,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.04593906666013953,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.05047880110050221,
 0,
 0,
 0.05111753054071422,
 0,
 0,
 0,
 0,
 0.053434114021872835,
 0,
 0,
 0,
 0,
 0,
 0.05488676455451426,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.05631199287510394,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.05980263473556123,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.06413449629312022,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.12949921325035596,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
